In [1]:
import psycopg2
import pandas as pd
import subprocess

Table creation test

In [2]:
#Creating a connection to the database

try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=password_you_created_for_PSQL")
except psycopg2.Error as e:
        print("Error: Could not make connection to the Postgres database")
        print(e)
        

In [3]:
#Create a cursor that can be used to execute queries

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")


In [4]:
#Set automatic commit to be true sp that each action is commited without having to call conn.commit() after each command

conn.set_session(autocommit=True)


In [5]:
#Create a database to do the work in

try: 
    cur.execute("create database myfirstdb")
except psycopg2.Error as e:
    print(e)
    

database "myfirstdb" already exists



In [6]:
#Add the database name in the connect statement. Lets close our connection to the default database,
#reconnect to the Udacity database, and get a new cursor

try: 
    conn.close()
except psycopg.Error as e:
    print(e)

try: #dbname=myfirstdb
    conn = psycopg2.connect("host=127.0.0.1 dbname=myfirstdb user=postgres password=Pol!2016")
except psycopg2.Error as e:
        print("Error: Could not make connection to the Postgres database")
        print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
    
conn.set_session(autocommit=True)
    

In [7]:
#Create table for students wich includes some columns

try:
    cur.execute("CREATE TABLE IF NOT EXISTS students (student_id int, name varchar, \
                age int, gender varchar, subject varchar, marks int);")
except psycopg2.Error as e:
    print("Error: issue creating table")
    print(e)


In [8]:
#Insert some rows into it

try: 
    cur.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                 VALUES (%s, %s, %s, %s, %s, %s)", \
                 (1, "Marcos", 23, "Male", "Python", 85))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                  VALUES (%s, %s, %s, %s, %s, %s)",
                  ( 2, "Marta", 22, "Female", "Python", 86))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


In [9]:
#Validate your data wasr inserted into table

try: 
    cur.execute("SELECT * FROM students;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    

(1, 'Marcos', 23, 'Male', 'Python', 85)
(2, 'Marta', 22, 'Female', 'Python', 86)


In [55]:
#Finally close your cursor and connection

cur.close()
conn.close()


-----

Put data into a new db

In [2]:
#Connection created

def connection(db_name, commit=False):
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname={} user=postgres password=Pol!2016".format(db_name))
    except psycopg2.Error as e:
            print("Error: Could not make connection to the Postgres database")
            print(e)
            
    #If a True were given to this function, all autocommits will turn true
    if commit == True:    
        conn.set_session(autocommit=True)
            
    return conn


In [3]:
#Create a cursor

def cursor(conn):
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Error: Could not get curser to the Database")
        print(e)
        
    return cur


In [4]:
#Conecting and cursor default

#Create a conection
conn_default = connection("postgres", True)

#Create a cursor
cur_default = cursor(conn_default)


In [6]:
#Create a new db

try: 
    cur_default.execute("create database internet_prices_22")
except psycopg2.Error as e:
    print(e)


In [7]:
#Close the default connection and cursor then recreated to created one

try: 
    conn_default.close()
    cur_default.close()
    
except psycopg.Error as e:
    print(e)

#dbname=internet_prices_22
#Create a new connection
conn = connection("internet_prices_22", True)

#Create a new cursor
cur = cursor(conn)



--------------

Treating with Pandas

In [84]:
#Get all data from a next folder

df_prices = pd.read_csv('internet_prices_22/prices.csv')
df_speed = pd.read_csv('internet_prices_22/speed.csv')
df_users = pd.read_csv('internet_prices_22/users.csv')


In [85]:
#Prices table

df_prices.head()


,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020)
0,IL,Israel,NEAR EAST,27.0,$0.05,$0.02,$20.95,$0.11,$0.90
1,KG,Kyrgyzstan,CIS (FORMER USSR),20.0,$0.15,$0.10,$7.08,$0.21,$0.27
2,FJ,Fiji,OCEANIA,18.0,$0.19,$0.05,$0.85,$0.59,$3.57
3,IT,Italy,WESTERN EUROPE,29.0,$0.27,$0.09,$3.54,$0.43,$1.73
4,SD,Sudan,SUB-SAHARAN AFRICA,33.0,$0.27,$0.03,$0.92,$0.63,$0.68


In [86]:
#Speed table

df_speed.head()


,Country,Avg \n(Mbit/s)Ookla
0,Canada,75.18
1,South Korea,117.95
2,Netherlands,108.33
3,Japan,44.05
4,Norway,134.73


In [87]:
#Speed's types are already ok

df_speed.dtypes


Country                 object
Avg \n(Mbit/s)Ookla    float64
dtype: object

In [88]:
#Users table

df_users.head()


,Country or area,Subregion,Region,Internet users,Population
0,China,Eastern Asia,Asia,"1,010,740,000","1,427,647,786"
1,India,Southern Asia,Asia,"833,710,000","1,352,642,280"
2,United States,Northern America,Americas,"312,320,000","327,096,265"
3,Indonesia,South-eastern Asia,Asia,"196,000,000","267,670,543"
4,Brazil,South America,Americas,"160,010,801","209,469,323"


In [89]:
#Useres's types are not ok - the two right columns should be integers

df_users.dtypes


Country or area    object
Subregion          object
Region             object
Internet users     object
Population         object
dtype: object

---

In [90]:
#There are values that are not numeric

df_prices['Average price of 1GB (USD)'].to_list()


['$0.05',
 '$0.15',
 '$0.19',
 '$0.27',
 '$0.27',
 '$0.29',
 '$0.32',
 '$0.34',
 '$0.38',
 '$0.39',
 '$0.41',
 '$0.42',
 '$0.43',
 '$0.43',
 '$0.49',
 '$0.51',
 '$0.52',
 '$0.56',
 '$0.59',
 '$0.59',
 '$0.60',
 '$0.60',
 '$0.60',
 '$0.61',
 '$0.63',
 '$0.64',
 '$0.66',
 '$0.68',
 '$0.70',
 '$0.74',
 '$0.75',
 '$0.75',
 '$0.75',
 '$0.76',
 '$0.76',
 '$0.78',
 '$0.79',
 '$0.81',
 '$0.82',
 '$0.83',
 '$0.83',
 '$0.85',
 '$0.88',
 '$0.88',
 '$0.88',
 '$0.89',
 '$0.89',
 '$0.90',
 '$0.92',
 '$0.94',
 '$0.94',
 '$0.96',
 '$0.97',
 '$1.02',
 '$1.04',
 '$1.06',
 '$1.06',
 '$1.08',
 '$1.09',
 '$1.09',
 '$1.12',
 '$1.13',
 '$1.14',
 '$1.15',
 '$1.17',
 '$1.18',
 '$1.23',
 '$1.24',
 '$1.25',
 '$1.28',
 '$1.28',
 '$1.30',
 '$1.32',
 '$1.33',
 '$1.38',
 '$1.39',
 '$1.41',
 '$1.42',
 '$1.42',
 '$1.45',
 '$1.46',
 '$1.47',
 '$1.51',
 '$1.51',
 '$1.52',
 '$1.56',
 '$1.56',
 '$1.58',
 '$1.61',
 '$1.67',
 '$1.67',
 '$1.70',
 '$1.71',
 '$1.71',
 '$1.72',
 '$1.76',
 '$1.77',
 '$1.80',
 '$1.80',
 '$1.82',


In [91]:
#Keep only values that seems to be numeric

df_prices = df_prices[~(df_prices['Average price of 1GB (USD)'].isin(['NO PROVIDERS','HYPERINFLATION',
                                                                         "Prices listed in non-convertible 'units'",
                                                                         'UNRELIABLE EXCHANGE RATES']))]


In [92]:
df_prices.head()

,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020)
0,IL,Israel,NEAR EAST,27.0,$0.05,$0.02,$20.95,$0.11,$0.90
1,KG,Kyrgyzstan,CIS (FORMER USSR),20.0,$0.15,$0.10,$7.08,$0.21,$0.27
2,FJ,Fiji,OCEANIA,18.0,$0.19,$0.05,$0.85,$0.59,$3.57
3,IT,Italy,WESTERN EUROPE,29.0,$0.27,$0.09,$3.54,$0.43,$1.73
4,SD,Sudan,SUB-SAHARAN AFRICA,33.0,$0.27,$0.03,$0.92,$0.63,$0.68


In [108]:
df_prices.columns

Index(['Country code', 'Name', 'Continental region', 'NO. OF Internet Plans ',
       'Average price of 1GB (USD)', 'Cheapest 1GB for 30 days (USD)',
       'Most expensive 1GB (USD)',
       'Average price of 1GB (USD  at the start of 2021)',
       'Average price of 1GB (USD – at start of 2020)'],
      dtype='object')

In [93]:
#Transform some columns

#Prices table
df_prices['Average price of 1GB (USD)']=df_prices['Average price of 1GB (USD)'].str.replace('$','0').astype(float)
df_prices['Cheapest 1GB for 30 days (USD)']=df_prices['Cheapest 1GB for 30 days (USD)'].str.replace('$','0').astype(float)
df_prices['Most expensive 1GB (USD)']=df_prices['Most expensive 1GB (USD)'].str.replace('$','0').astype(float)
df_prices['Average price of 1GB (USD  at the start of 2021)']=df_prices['Average price of 1GB (USD  at the start of 2021)'].str.replace('$','0').replace('NO PACKAGES','0').astype(float)
df_prices['Average price of 1GB (USD – at start of 2020)']=df_prices['Average price of 1GB (USD – at start of 2020)'].str.replace('$','0').replace('NO PACKAGES','0').astype(float)

df_prices['NO. OF Internet Plans '] = df_prices['NO. OF Internet Plans'].astype(int) 

#Users table
df_users['Internet users']=df_users['Internet users'].str.replace(',','').astype(int)
df_users['Population']=df_users['Population'].fillna('0')
df_users['Population']=df_users['Population'].str.replace(',','').astype(int)


/tmp/ipykernel_234/3679330471.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_prices['Average price of 1GB (USD)']=df_prices['Average price of 1GB (USD)'].str.replace('$','0').astype(float)
/tmp/ipykernel_234/3679330471.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_prices['Cheapest 1GB for 30 days (USD)']=df_prices['Cheapest 1GB for 30 days (USD)'].str.replace('$','0').astype(float)
/tmp/ipykernel_234/3679330471.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_prices['Most expensive 1GB (USD)']=df_p

In [110]:
#Save dfs as csv

df_prices.to_csv("internet_prices_22/prices.csv", index=False)
df_users.to_csv("internet_prices_22/users.csv", index=False)


In [111]:
#Move edited files to the Public folder

subprocess.call("./move.sh")
print('Files copied')


Files copied


--------

In [104]:
#Create tables

try:
    #Prices
    cur.execute("CREATE TABLE prices (country_code varchar, name varchar, \
                region varchar, packs_available int, avg_price_1GB decimal, min_price_1GB decimal, \
                max_price_1GB decimal, avg_price_1GB_2021 decimal, avg_price_1GB_2020 decimal);")
    
    #Speed
    cur.execute("CREATE TABLE speed (country varchar, avg_mbitPerSec decimal);")
    
    #Users
    cur.execute("CREATE TABLE users (country varchar, subregion varchar, region varchar, \
                internet_users int, population int);")
    
except psycopg2.Error as e:
    print("Error: issue creating table")
    print(e)
    

In [112]:
#Import data from files to tables

try: 
    cur.execute("COPY prices(country_code, name, \
                region, packs_available, avg_price_1GB, min_price_1GB, \
                max_price_1GB , avg_price_1GB_2021, avg_price_1GB_2020) \
                FROM 'C:\\Users\\Public\\internet_prices_22\\prices.csv' \
                DELIMITER ',' \
                CSV HEADER;")
    
    cur.execute("COPY speed(country, avg_mbitPerSec) \
                FROM 'C:\\Users\\Public\\internet_prices_22\\speed.csv' \
                DELIMITER ',' \
                CSV HEADER;")
    
    cur.execute("COPY users(country, subregion, region, \
                internet_users, population) \
                FROM 'C:\\Users\\Public\\internet_prices_22\\users.csv' \
                DELIMITER ',' \
                CSV HEADER;")
    
except psycopg2.Error as e: 
    print (e)

#Important note:
#I have to move internet_prices_22 folder to the Public one. Because Postgres does not get permission to read any file instead
#it is there


---

In [114]:
#Check tables from Notebook

cur.execute("SELECT * FROM prices limit 5;")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


('IL', 'Israel', 'NEAR EAST', 27, Decimal('0.05'), Decimal('0.02'), Decimal('20.95'), Decimal('0.11'), Decimal('0.9'))
('KG', 'Kyrgyzstan', 'CIS (FORMER USSR)', 20, Decimal('0.15'), Decimal('0.1'), Decimal('7.08'), Decimal('0.21'), Decimal('0.27'))
('FJ', 'Fiji', 'OCEANIA', 18, Decimal('0.19'), Decimal('0.05'), Decimal('0.85'), Decimal('0.59'), Decimal('3.57'))
('IT', 'Italy', 'WESTERN EUROPE', 29, Decimal('0.27'), Decimal('0.09'), Decimal('3.54'), Decimal('0.43'), Decimal('1.73'))
('SD', 'Sudan', 'SUB-SAHARAN AFRICA', 33, Decimal('0.27'), Decimal('0.03'), Decimal('0.92'), Decimal('0.63'), Decimal('0.68'))
